# Test Bed Sliders

Import the test bed from the previous module into a new coordinate system

In [1]:
import bumblebee as bb

app = bb.Application()
assm = app.csys
assm.ax.set_xlim([-20,20])
assm.ax.set_ylim([-20,20])
assm.ax.set_zlim([-10,15])
assm.plot()

test_bed = bb.RigidCollection.from_json('TestBedAssm.json')
assm.add(test_bed)

### Set up body variables

Pull out useful bodies from the part tree so that they can be called by name. 

`TODO make a method to find the node by name so that this process isn't so clunky.`

In [2]:
arm=None
slide_assm=None
gantry=None
stewart=None
bplate=None
tplate=None

for node in test_bed.nodes:
    arm = node if node.name == 'Arm' else arm
    slide_assm = node if node.name == 'SlideAssm' else slide_assm
    
for node in slide_assm.nodes:
    gantry = node if node.name == 'Gantry' else gantry
    stewart = node if node.name == 'StewartAssm' else stewart
    
for node in stewart.nodes:
    bplate = node if node.name == 'BottomPlateAssm' else bplate
    tplate = node if node.name == 'TopPlateAssm' else tplate

### Set up Jupyter Sliders

In [3]:
import ipywidgets as widgets

In [4]:
gantry, stewx, stewy, stewz, rotx, roty, rotz = [widgets.FloatSlider(
    value=0,
    min=-10.0,
    max=10.0,
    step=0.1,
    description=f'{desc}:',
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
) for desc in ['Gantry', 'StewX', 'StewY', 'StewZ', 'RotX', 'RotY', 'RotZ']]

In [5]:
widgets.VBox([app.csys.plot(), gantry, stewx, stewy, stewz, rotx, roty, rotz])

### Stewart Kinematics

In [6]:
bottom_joints = []
for node in bplate.nodes:
    if "Bracket" in node.name:
        for subnode in node.nodes:
            if "Joint" in subnode.name:
                bottom_joints.append(subnode)

In [7]:
top_joints = []
for node in tplate.nodes:
    if "Bracket" in node.name:
        for subnode in node.nodes:
            if "Joint" in subnode.name:
                top_joints.append(subnode)   

In [8]:
import numpy as np
links = []
for bj, tj in zip([0,1,2,3,4,5],[2,5,0,3,4,1]):
    start = top_joints[tj].tf[0:3,3]
    end = bottom_joints[bj].tf[0:3,3]
    
    comb = np.vstack((start,end)).T
    
    link, = assm.ax.plot(*comb, 'r')
    links.append(link)

### Implement Slider Updates

In [9]:
def update_links():
    for i, (bj, tj) in enumerate(zip([0,1,2,3,4,5],[2,5,0,3,4,1])):
        start = top_joints[tj].tf[0:3,3]
        end = bottom_joints[bj].tf[0:3,3]

        comb = np.vstack((start,end)).T

        links[i]._verts3d = (comb[0], comb[1], comb[2])

def update_gantry(change):
    dx =change.new-change.old
    slide_assm.translate([dx,0,0])
    update_links()
    
def update_stewx(change):
    dx =change.new-change.old
    bplate.translate([dx,0,0])
    update_links()
    
def update_stewy(change):
    dy =change.new-change.old
    bplate.translate([0,dy,0])
    update_links()
    
def update_stewz(change):
    dz =change.new-change.old
    bplate.translate([0,0,dz])
    update_links()
    
def update_rotx(change):
    dx =change.new-change.old
    bplate.rotate_from_euler('x', dx, degrees=True)
    update_links()
    
def update_roty(change):
    dy =change.new-change.old
    bplate.rotate_from_euler('y', dy, degrees=True)
    update_links()
    
def update_rotz(change):
    dz =change.new-change.old
    bplate.rotate_from_euler('z', dz, degrees=True)
    update_links()

In [10]:
gantry.observe(update_gantry, names='value')
stewx.observe(update_stewx, names='value')
stewy.observe(update_stewy, names='value')
stewz.observe(update_stewz, names='value')
rotx.observe(update_rotx, names='value')
roty.observe(update_roty, names='value')
rotz.observe(update_rotz, names='value')

In [11]:
# stewart.toggle_vis()
# for joint in top_joints+bottom_joints:
#     joint.toggle_vis()